<a href="https://colab.research.google.com/github/Stephanie9606/zeno_dashboard_nlp/blob/main/KHC_News_JanAug_Key_Sent_0915.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **KHC Dashboard Pulsar Data Testing File - Sentiment + Keywords**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data cleaning

--- KHC Data Jan - Jun 2023 ---

In [ ]:
khc_df_16 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/khc_H1_newsblogs_clean_nodirt.xlsx')

In [ ]:
# only keep 'news' (rm blogs)
khc_df_16 = khc_df_16[khc_df_16['source'] == 'Online News']

--- KHC Data July - Aug 2023 ---

In [ ]:
khc_df_78 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/khc_news_JulyAug.xlsx')

In [ ]:
# merge two data
khc_df = pd.concat([khc_df_16, khc_df_78], axis=0)

In [ ]:
# check num of rows
khc_df.shape[0]

16297

In [ ]:
# rm rows with duplicated contents
khc_df = khc_df[~khc_df.duplicated(subset='content', keep=False)]

In [ ]:
khc_df.shape[0]

13006

In [ ]:
# keep only relevant columns
columns_to_keep = [0, 1, 2, 5, 6, 9, 14, 18]
khc_df = khc_df.iloc[:, columns_to_keep]

In [ ]:
# run pulsar query again, extract relevant content (security -> food security, too many noise) (Delete "Heinz" in the News!)
keywords = ["Kraft Heinz", "KHC", "Kraft Mac and Cheese", "Kraft Singles", "Miracle Whip", "Heinz ketchup", "Lunchables", "Velveeta"]
topics = ["supply chain", "supply", "food security", "supplier", "recycled", "recycle", "recycling", "single-use", "plastic", "package", "packaging", "waste", "environment", "nature", "product", "quality", "planet", "crisis", "challenge", "problem", "criticism", "reputation"]

mask = khc_df['content'].str.contains('|'.join(keywords), case=False) & khc_df['content'].str.contains('|'.join(topics), case=False)

khc_df = khc_df[mask]

In [ ]:
khc_df.shape[0]

8866

In [ ]:
khc_df.reset_index(drop=True, inplace=True)

--- rm content with high similarity: 90% ---

In [ ]:
# break into each df 100 rows
# number of df going to break into
num_splits = len(khc_df) // 100 + 1

# create a dictionary to store the split DataFrames
split_dfs = {}

# split the original DataFrame into smaller DataFrames and store them in the dictionary
for i in range(num_splits):
    start_idx = i * 100
    end_idx = (i + 1) * 100
    df_name = f'df_{i + 1}'
    split_dfs[df_name] = khc_df.iloc[start_idx:end_idx]

# access the split DataFrames by name:
#print(split_dfs['df_1'])

In [ ]:
# iterate through the split DataFrames and reset the indices
for df_name, df in split_dfs.items():
    split_dfs[df_name] = df.reset_index(drop=True)

In [ ]:
# take a look
#split_dfs['df_2']

--- takes around 5-6 mins ---

In [ ]:
from difflib import SequenceMatcher

# Define the similarity threshold
similarity_threshold = 0.9

# Define a function to process a DataFrame
def process_dataframe(df):
    # Create a list to store first sentences that have been seen
    seen_first_sentences = []

    # Create a set to store the indices of rows to remove
    indices_to_remove = set()

    # Define a function to extract the first sentence from a string
    def extract_first_sentence(text):
        sentences = text.split('.')
        if sentences:
            return sentences[0]
        else:
            return text

    # Iterate through the DataFrame
    for index, row in df.iterrows():
        content = row['content']  # Replace 'data' with the actual column name in your DataFrame
        first_sentence = extract_first_sentence(content)

        # Check if a similar first sentence has been seen before
        similar_to_seen = any(
            similarity >= similarity_threshold
            for similarity in [SequenceMatcher(None, first_sentence, seen).ratio() for seen in seen_first_sentences]
        )

        if not similar_to_seen:
            seen_first_sentences.append(first_sentence)
        else:
            indices_to_remove.add(index)

    # Remove rows with similar first sentences from the current DataFrame
    df = df.drop(list(indices_to_remove))

    # Reset the index of the current DataFrame
    df = df.reset_index(drop=True)

    return df

# Iterate through the DataFrames in split_dfs and process each one
for df_name in split_dfs.keys():
    split_dfs[df_name] = process_dataframe(split_dfs[df_name])

# You now have processed DataFrames back in the original dictionary split_dfs

In [ ]:
# concat all DataFrames into one big DataFrame
big_df = pd.concat(split_dfs.values(), ignore_index=True)

# final length of the df
big_df.shape[0]

5157

In [ ]:
### GOOD TO GO --> split to eaxh 100 rows and run then concat them
from difflib import SequenceMatcher

# Define the similarity threshold
similarity_threshold = 0.9

# Create a list to store first sentences that have been seen
seen_first_sentences = []

# Create a set to store the indices of rows to remove
indices_to_remove = set()

# Define a function to extract the first sentence from a string
def extract_first_sentence(text):
    sentences = text.split('.')
    if sentences:
        return sentences[0]
    else:
        return text

# Iterate through the DataFrame
for index, row in khc_df_test.iterrows():
    content = row['content']
    first_sentence = extract_first_sentence(content)

    # Check if a similar first sentence has been seen before
    similar_to_seen = any(
        similarity >= similarity_threshold
        for similarity in [SequenceMatcher(None, first_sentence, seen).ratio() for seen in seen_first_sentences]
    )

    if not similar_to_seen:
        seen_first_sentences.append(first_sentence)
    else:
        indices_to_remove.add(index)

# Remove rows with similar first sentences
khc_df_test = khc_df_test.drop(list(indices_to_remove))

# Reset the index
khc_df_test = khc_df_test.reset_index(drop=True)

# Print the resulting DataFrame
print(khc_df_test)

In [ ]:
# save to new excel file
big_df.to_excel('/content/drive/MyDrive/Zeno_KHC_News/khc_news_JanAug_clean.xlsx', index=False)

In [ ]:
big_df.head(10)

# Directly read cleaned file

In [ ]:
# 5157 rows
khc_df_news = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/khc_news_JanAug_clean.xlsx')

In [ ]:
khc_df_news.shape[0]

5157

# Keywords: KeyBert

--- every 1000 rows takes approximately  mins ---

In [ ]:
# split into 5 df, each data has about 1000 rows
khc_df_n1 = khc_df_news.iloc[:500,:]

In [ ]:
khc_df_n2 = khc_df_news.iloc[500:1000,:]

In [ ]:
khc_df_n3 = khc_df_news.iloc[1000:1500,:]

In [ ]:
khc_df_n4 = khc_df_news.iloc[1500:2000,:]

In [ ]:
khc_df_n5 = khc_df_news.iloc[2000:2500,:]

In [ ]:
khc_df_n6 = khc_df_news.iloc[2500:3000,:]

In [ ]:
khc_df_n7 = khc_df_news.iloc[3000:3500,:]

In [ ]:
khc_df_n8 = khc_df_news.iloc[3500:4000,:]

In [ ]:
khc_df_n9 = khc_df_news.iloc[4000:4500,:]

In [ ]:
khc_df_n10 = khc_df_news.iloc[4500:,:]

In [ ]:
#!pip3 install keybert
from keybert import KeyBERT

In [ ]:
#!pip3 install keyphrase-vectorizers
#from keyphrase_vectorizers import KeyphraseCountVectorizer

In [ ]:
#!pip3 install transformers
# init default vectorizer
#vectorizer = KeyphraseCountVectorizer()
#print(vectorizer.get_params()) # print parameters

# specify model
kw_model = KeyBERT('all-MiniLM-L12-v2')
# specify number of keywords to extract
n_keywords = 5
ngram = 2  # specify ngram of keywords

In [ ]:
# function to extract keywords for a batch of rows
def get_keywords_batch(rows):
    contents = rows['content'].tolist()
    keywords_batch = kw_model.extract_keywords(contents, stop_words='english', highlight=False, top_n=n_keywords, keyphrase_ngram_range=(1, ngram), diversity=0.2) # vectorizer=vectorizer
    return [[keyword[0] for keyword in keywords] for keywords in keywords_batch]

In [ ]:
### change n1 and n2 df to proceed
# split the data into batches (higher effieciency)
batch_size = 100
batches = [khc_df_n10[i:i+batch_size] for i in range(0, len(khc_df_n10), batch_size)]

--- takes approximately 25 mins each 500 rows ---

In [ ]:
# extract keywords for each batch and concatenate the results
keyword_lists = []
for batch in batches:
    keyword_lists.extend(get_keywords_batch(batch))

In [ ]:
### change n1 and n2 df to proceed
# add the keyword lists as a new column to the DataFrame
khc_df_n10['keywords'] = keyword_lists
# format to better fit in Excel
khc_df_n10['keywords'] = khc_df_n10['keywords'].apply(lambda x: ', '.join(x))

<ipython-input-58-f43bdaae4a20>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_n10['keywords'] = keyword_lists
<ipython-input-58-f43bdaae4a20>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  khc_df_n10['keywords'] = khc_df_n10['keywords'].apply(lambda x: ', '.join(x))


In [ ]:
khc_df_n2.iloc[:,[3,-2,-1]]

In [ ]:
### change n1 and n2 df to proceed
khc_df_n10.to_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_10.xlsx', index=False)

--- Read all the files with keywords ---

In [ ]:
# read all the files
khc_df_n1 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_1.xlsx')
khc_df_n2 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_2.xlsx')
khc_df_n3 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_3.xlsx')
khc_df_n4 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_4.xlsx')
khc_df_n5 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_5.xlsx')
khc_df_n6 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_6.xlsx')
khc_df_n7 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_7.xlsx')
khc_df_n8 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_8.xlsx')
khc_df_n9 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_9.xlsx')
khc_df_n10 = pd.read_excel('/content/drive/MyDrive/Zeno_KHC_News/News_w_Keywords0915/khc_news_JanAug_key5gram2_0921_10.xlsx')

In [ ]:
# merge n1 - n10
khc_news_key = pd.concat([khc_df_n1, khc_df_n2, khc_df_n3, khc_df_n4, khc_df_n5, khc_df_n6, khc_df_n7, khc_df_n8, khc_df_n9, khc_df_n10], axis=0)

In [ ]:
khc_news_key.head()

# Add Category

In [ ]:
# define keyword categories
keyword_categories = {
    "[Key Issue] Supply chain": ["supply chain", "supply", "supplier"],
    "[Key Issue] Single-use plastic": ["recycle", "single-use", "plastic", "package", "packaging"],
    "[Key Issue] Sustainability": ["waste", "environment", "nature", "planet"],
    "FDA leadership": ["leadership", "FDA"],
    "Food safety": ["food safety"],
    "Crisis & Criticism": ["crisis", "challenge", "problem", "criticism"],
    "Food security": ["food security", "security"],
    "Brand reputation": ["reputation"],
    "Product quality": ["product", "quality"],
}

# Function to count keyword occurrences in a given text and return the category
def categorize_keywords(text):
    keyword_counts = {category: 0 for category in keyword_categories}
    text = text.lower()  # Convert text to lowercase for case-insensitive counting

    for category, keywords in keyword_categories.items():
        for keyword in keywords:
            keyword_counts[category] += text.count(keyword)

    # Check if all counts are zero and assign a special category for no keywords found
    if all(count == 0 for count in keyword_counts.values()):
        return "Others"

    return max(keyword_counts, key=keyword_counts.get)

khc_news_key['category'] = khc_news_key['content'].apply(categorize_keywords)

In [ ]:
khc_news_key.head()

In [ ]:
khc_news_key.to_excel('/content/drive/MyDrive/Zeno_KHC_News/khc_news_JanAug_all_0925.xlsx', index=False)

# Sentiment using DistillBERT for News

# Check sentiment quality

In [ ]:
# check unique output
khc_df_n1000['flair_sentiment'].unique()

array(['negative', 'positive'], dtype=object)

In [ ]:
# check on neutral prediction
khc_df_n1000['flair_sentiment'].str.contains('neutral', case=False).any()

False

In [ ]:
neutral_df = khc_df_n1000[khc_df_n1000['flair_sentiment'] == 'neutral']
neutral_df.head()

,id,search,source,application,title,content,date,parent,language,url,...,human rights,labor practices,community and society,workforce protection,local content,post subtype,post type,keywords,flair_sentiment,flair_sentiment_score


In [ ]:
# check the score
low_df = khc_df_tw[khc_df_tw['flair_sentiment_score'] < 0.7]
low_df.shape[0]

182

In [ ]:
low_df.iloc[:,[5,-2,-1]]

,content,flair_sentiment,flair_sentiment_score
14,Buffett praising $COST's Kirkland brand. He st...,positive,0.500105
17,It's your problem you took it any other way!...,negative,0.657860
43,Lord Cornwallis: {{short description|1955 film...,positive,0.610610
84,"Heinz say:Because of its natural acidity, Hei...",positive,0.603075
91,Today on Garret Lewis i spoke about happened a...,negative,0.690468
...,...,...,...
2921,Sometimes! If it is quality stuff.Not Heinz.,negative,0.527366
2950,This looks like a food security marker.Heinz ...,negative,0.562701
2954,"Freelance creative director, Elliott Starr, wa...",negative,0.637750
2988,For real. You'd have to challenge in court (...,negative,0.585367


--- if sentiment score < 0.7 --> set to neutral ---

In [ ]:
#khc_df_tw.loc[khc_df_tw['flair_sentiment_score'] < 0.7, 'flair_sentiment'] = 'neutral'

# Difference between Pulsar and Flair sentiment

In [ ]:
# Twitter
different_df = khc_df_n1000[khc_df_n1000['flair_sentiment'] != khc_df_n1000['sentiment class']]
different_df = different_df.iloc[:, [0,5,18,-2,-1]]
different_df

# Sum: BART

In [ ]:
#!pip3 install transformers
from transformers import BartTokenizer, BartForConditionalGeneration
import tensorflow as tf

In [ ]:
model_name = "facebook/bart-large-xsum" #"facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# define a function to generate summaries using BART for a batch of texts
def generate_summaries_batch(texts):
    inputs = tokenizer.batch_encode_plus(texts, return_tensors="pt", max_length=1024, truncation=True, padding=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=100, min_length=30, length_penalty=2.0, num_beams=6, no_repeat_ngram_size=2, early_stopping=True)
    summaries = [tokenizer.decode(summary, skip_special_tokens=True) for summary in summary_ids]
    return summaries

In [ ]:
khc_sum_n100 = khc_df_news.iloc[:100,:]

In [ ]:
khc_sum_n100.iloc[:,[5,-1]]

In [ ]:
# batch size for processing multiple texts at once
batch_size = 25

# create batches of texts for summarization
text_batches = [khc_sum_n100["content"][i:i+batch_size].tolist() for i in range(0, len(khc_sum_n100), batch_size)]

# generate summaries for each batch and concatenate the results
summaries = []
for batch in text_batches:
    batch_summaries = generate_summaries_batch(batch)
    summaries.extend(batch_summaries)

KeyboardInterrupt: ignored

In [ ]:
# Add the summaries to the DataFrame
khc_sum_n100["bart_summary"] = summaries

In [ ]:
khc_sum_n100.iloc[:,[5,-1]]

In [ ]:
# save to new excel file
khc_sum_n100.to_excel('/content/drive/MyDrive/Zeno/news_w_sum/khc_H1_khc_n100_BARTxsum.xlsx', index=False)

# Export the data to an excel file

In [ ]:
# move 'keywords' ahead follow by 'content'
columns = khc_df_tw.columns.tolist()
columns.insert(6, columns.pop(-3))
khc_df_tw = khc_df_tw[columns]

In [ ]:
khc_df_tw.head()

,id,search,source,application,title,content,keywords,date,parent,language,...,biodiversity,human rights,labor practices,community and society,workforce protection,local content,post subtype,post type,flair_sentiment,flair_sentiment_score
0,0_104815_8_1609539758241484801,104815,Twitter,Twitter for iPad,NaN,"You mean ""Catsup."" Seriously, Hunts had a br...","ketchup, catsup, heinz trademarks, heinz, hunt...",2023-01-01 13:19:16,0_104815_8_1609425583464943624,en,...,NaN,NaN,NaN,NaN,NaN,NaN,reply,engagement,negative,0.759982
1,0_104815_8_1609589331206836224,104815,Twitter,Twitter for iPhone,NaN,I smell a charity stream here…Each mileston...,"heinz beans, different heinz beans product, he...",2023-01-01 16:36:15,0_104815_8_1609588968877883393,en,...,NaN,NaN,NaN,NaN,NaN,NaN,reply,engagement,positive,0.952374
2,0_104815_8_1609705628557275136,104815,Twitter,Twitter for iPhone,NaN,if anyone wants to get fired up ask them abou...,"heinz plastic, heinz, kit, plastic, anyone",2023-01-02 00:18:23,0,en,...,NaN,NaN,NaN,NaN,NaN,NaN,original post,post,positive,0.927865
3,0_104815_8_1609732418335899651,104815,Twitter,Twitter for Android,NaN,"just so you know, I work in a supermarket and ...","plastic free packaging, heinz beans, packaging...",2023-01-02 02:04:50,0,en,...,NaN,NaN,NaN,NaN,NaN,NaN,original post,post,negative,0.988997
4,0_104815_8_1609742063314309120,104815,Twitter,Twitter Web App,NaN,So hard to believe Velveeta Voldemort suppo...,"kevin mccarthy, velveeta voldemort, own party,...",2023-01-02 02:43:10,0_104815_8_1609735600881504257,en,...,NaN,NaN,NaN,NaN,NaN,NaN,reply,engagement,negative,0.937185


In [ ]:
# save to new excel file
khc_df_tw.to_excel('/content/drive/MyDrive/Zeno/khc_H1_updated0804.xlsx', index=False)